<a href="https://colab.research.google.com/github/Maryamdeveloper/python-simulation/blob/main/linearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load dataset
import pandas as pd
df = pd.read_excel('/content/drive/My Drive/Copy_patients.xlsx')

In [ ]:
# change categorical data to numerical
df['Status'].replace(['Excellent','Good','Fair','Poor'],[4,3,2,1],inplace=True);
df=pd.get_dummies(df,columns=['Gender','Smoker'],drop_first=True);   #changed the gender and smoker to two boolean type

In [ ]:
# calculate correlation matrix for data columns
Corr=df.corr()
# correlation value is between 0-1 and 1 means max correlation positive

In [ ]:
# response variable
y=df['Systolic'].values

In [ ]:
# predictor value
x=df[['Age','Height','Weight','Gender_Male','Smoker_True']].values

# here linear regression with scikit learn starts


In [ ]:
from sklearn.linear_model import LinearRegression
# define model
model=LinearRegression()
# means train the model and get the regression parameters value
# note: here fit_intercept is the b0 that was const coef in our equation which by default is true
model.fit(x,y)
# r2 is calculated which was between 0-1 which 1 meant very good fitted and 0 meant chance level and output is const
model.score(x,y)
# see the coefficient of equation that was given to each predictor parameter that we had to compare these coefficient it is better that the data had been normalized
coef=model.coef_
# see the const coefficient of the equation
const_coef=model.intercept_

In [ ]:
# predict the response value we wanted to predict
y_pred=model.predict(x)

In [ ]:
# now that we have the pred value we can calculate r2 in another way
from sklearn.metrics import r2_score,mean_squared_error
r2_score(y,y_pred)
# means if we y-ypred for each row then we raised it by power of 2 and calculate the mean of it how much is that this is  also another measure to evaluate the model but r2 is better
mean_squared_error(y,y_pred)

21.593096827011642

#### if we want to compare model coefficients we should normalize it or do standardization on it we have 2 important type here


In [ ]:
# 1.zscore: for each column mean of number is 0 and standard derivation 1 and the values become between -1,1 and they can be even more than that when it is a greater value, in some resources this is calld standardization
import numpy as np
from scipy.stats import zscore
import copy
#to create a copy of data & put it in another variable
x2=copy.copy(x)
# note: we would not write x2=x because they will become 1 and point at the same point of memory both and changing one can change the other simulatniously but we dont want to do that
x_new = np.array(x, dtype=int) #add this due to TypeError: loop of ufunc does not support argument 0 of type float which has no callable sqrt method
x=zscore(x_new) #by default it will do it on columns but if we want to do zscore operation on rows we add axis=1 param to input of zscore()
model=LinearRegression()
model.fit(x,y)
model.score(x,y)
coef2=model.coef_ #values has changed but tehy keep themself meaning that value 4 is still the highest among the coefficients and means it still is more important & has more effects on patient

In [ ]:
# 2.minmaxscaler: max of each column would be 1 and min 0 mean that the values in each column would become between 0-1, in some resources this is calld normalization
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
# first we should fit this function on data then transform to max 1 min 0 use ; between them is like adding an enter mean that it should be run after it
scaler.fit(x);x=scaler.transform(x)
model=LinearRegression()
model.fit(x,y)
model.score(x,y)
coef3=model.coef_
# note: neither standardization nor normalization have no effect on the r2 score of our model but it can help us compare coefficients better

######to improve the r2 value we can use polynomial features which creates new reponse variable for us by combination of  predictor values that we have meaning that it uses the following equation:
𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²

In [ ]:
x=copy.copy(x2)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import numpy as np
# degree of predictor variables power in max state we want to have a loop on it
degrees=[1,2,3,4,5,6,7,8,9] #also we could have written range(1,10)
# note that this will create an power of 2 for your coeff and also multiply them
for i in range(len(degrees)): #also we can wite range 9 which means that it would iterate 9 times
  # degree max power of var means that for each times the degree would be i-th item of array
  # include_bias is false by default
  polynomial_features=PolynomialFeatures(degree=degrees[i],include_bias=False)
  model=LinearRegression()
  # pipeline is imported becausde our work has 2 steps 1-create polynomial featurs 2-linear regression is done
  model2=Pipeline([("polynomial_features",polynomial_features),("linear_regression",model)])
  model2.fit(x,y)
  print('degree: ',degrees[i])
  # number of features is the number of predictor variables that our model has created then had done regression using them
  print('number of features: ',model.n_features_in_)
  print('r2_score: ',model2.score(x,y))
  # after degree 5 it had been saturated and did not get higher

degree:  1
number of features:  5
r2_score:  0.5172329717745202
degree:  2
number of features:  20
r2_score:  0.584596961222823
degree:  3
number of features:  55
r2_score:  0.681249185894901
degree:  4
number of features:  125
r2_score:  0.7536088776122523
degree:  5
number of features:  251
r2_score:  0.7524710530766483
degree:  6
number of features:  461
r2_score:  0.7552591871099217
degree:  7
number of features:  791
r2_score:  0.7552903529824331
degree:  8
number of features:  1286
r2_score:  0.7552903529824331
degree:  9
number of features:  2001
r2_score:  0.7552903529824331


In [ ]:
# without loop
polynomial_features=PolynomialFeatures(degree=5,include_bias=False)
model=LinearRegression()
model2=Pipeline([("polynomial_features",polynomial_features),("linear_regression",model)])
# note that here after we fit x we had also predict it with x too meaning that it had been trained and tested with the same value that we had do this alot in regression
model2.fit(x,y)
y_pred=model2.predict(x)

In [ ]:
# to test if it is overfitting we use cross-validation
from sklearn.model_selection import cross_val_score
import numpy as np

degrees=[1,2,3,4,5,6,7,8,9]
for i in range(len(degrees)):
  polynomial_features=PolynomialFeatures(degree=degrees[i],include_bias=False)
  model=LinearRegression()
  pipeline=Pipeline([("polynomial_features",polynomial_features),("linear_regression",model)])
  print('degree: ',degrees[i])
  # model that we want to do cross validation on it, data, optimal output of data, scoring method, cv means that divide x data into 3 parts then do the training and test on it 3 times each time one of those 3 parts is chosen as test and use the other 2 to train the model
  scores=cross_val_score(pipeline,x,y,scoring="r2",cv=3)
  # this cv=3 will gives us 3 scores then we calculat ethe mean of them
  print('r2_score: ',np.mean(scores))
  # this - in results means that we have overfitting in our data
  # note that creating new predictor variables using polynomial might lead to overfitting when our data is so small!

degree:  1
r2_score:  0.41786302682533
degree:  2
r2_score:  0.18611693144479893
degree:  3
r2_score:  -2.2064218607214187e+25
degree:  4
r2_score:  -7.85934313714828e+23
degree:  5
r2_score:  -0.037720867226193756
degree:  6
r2_score:  -4.374450294236735e+21
degree:  7
r2_score:  -3.7395626773552067e+22
degree:  8
r2_score:  -5.1480872668670047e+26
degree:  9
r2_score:  -5.5832503326419785


# here linear regression with statsmodel starts


In [ ]:
!pip install statsmodels

In [ ]:
# the difference is that it will give you more parameters & values in its output and with higher accuracy but we can't use polynomial feature or use it in for loop
import statsmodels.api as sm
import numpy as np
# this will add an constant column to x which means that this library doesn't has the const coef by default so we need to add this to our data
x=sm.add_constant(x)
# ols->ordinary list square will do regression for us
model=sm.OLS(y,x)
# fit does not get any input
results=model.fit()
# can see r2
print(results.rsquared)
# we can observe that both of this libraries will give us the same result for r2 and they are working just like each other

0.5172329717745202


In [ ]:
# it will give you in details what had your regression model done with this library
# Adj. R-squared says that if 2 model has equal r2 the model that is better has less predictor variables will get higher adjust_r2
# P>|t| is equal to p-value it says that those predictor variables that their p-value is less than 0.05 or 0.001 has an impactful meaning in our output so that we can see except the const x5 which was smoker has less p-value or x4 has a large number which means that it has no effect on the model
# it says that in our data there is a large correlation between variables to check on this multicollinearity we go to corr that we had built earlier
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.517
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     20.14
Date:                Mon, 27 May 2024   Prob (F-statistic):           1.32e-13
Time:                        15:17:31   Log-Likelihood:                -295.38
No. Observations:                 100   AIC:                             602.8
Df Residuals:                      94   BIC:                             618.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        122.7800      0.479    256.507      0.0

In [ ]:
# we can observe 2thing we wanted the model to predict the systolic blood pressure from regression model and we use smoker as a predictor variable
# we saw it has 0.7 correlation with systolic which is high thats why it gets a higher coef and lower p-value
# on the other side we can see that gender and weight has 0.9 correlation with each other which means they have the same effect that are not predicators
# it is here that correlation matrix will help us in feature selection when we have a wide variety of features and use it in regression model preferably predicator variables like x1,x2,x3 have no correlation with each other and be independant
# also does'nt have high correlation with output
Corr

,Age,Height,Weight,Systolic,Diastolic,Status,Gender_Male,Smoker_True
Age,1.000000,0.116002,0.091356,0.134127,0.080597,0.072022,0.083279,0.060220
Height,0.116002,1.000000,0.695970,0.214076,0.156819,0.019152,0.722073,0.199114
Weight,0.091356,0.695970,1.000000,0.155788,0.222687,0.062000,0.945034,0.215586
Systolic,0.134127,0.214076,0.155788,1.000000,0.511843,0.054892,0.160008,0.706323
Diastolic,0.080597,0.156819,0.222687,0.511843,1.000000,0.043437,0.217507,0.723365
Status,0.072022,0.019152,0.062000,0.054892,0.043437,1.000000,-0.012264,0.000438
Gender_Male,0.083279,0.722073,0.945034,0.160008,0.217507,-0.012264,1.000000,0.212327
Smoker_True,0.060220,0.199114,0.215586,0.706323,0.723365,0.000438,0.212327,1.000000
